In [50]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import tensorflow as tf # 1.15.0 버전 사용
import os
import pandas as pd
import numpy as np
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [56]:
path = r"C:\Users\oym91\OneDrive\Desktop\graph_all\\"
trainFileList = os.listdir(path+'train')
testFileList = os.listdir(path+'test')

In [57]:
trainFileList = [x for x in trainFileList if 'png' in x]
testFileList = [x for x in testFileList if 'png' in x]

In [58]:
word = {'balance' : 0, 'decrease' : 1, 'increase' : 2, 'shortcycle' : 3}

In [59]:
# np.array에 각 데이터를 넣어주는 작업
for i, file in enumerate(trainFileList):
    #label, nm, tr, _ = file.split('_') # 예시 aa_bc_01_02.jpg
    label, _ = file.split('_')
    code, ext = _.split('.') # 예시 02.jpg

    # 이미지 불러오기
    new_img = load_img(path+'train\\'+file)
    # 이미지를 array 형식으로 변형
    arr_img = img_to_array(new_img)
    # (1, 28, 28, 3) 검정색 글씨뿐이지만 나름 RGB...zz
    img = arr_img.reshape((1,)+arr_img.shape)

    # 첫 번째 데이터의 경우 container를 생성
    if i == 0 :  # train 데이터
        container = img
        labels = word[label]
    # 첫 번쨰가 아닌경우 array에 계속 쌓아나감
    else:
        container = np.vstack([container, img])
        labels = np.vstack([labels, word[label]])

xTrain = container
# 카테고리 데이터 one-hot encoding
yTrain = np_utils.to_categorical(labels,5)

# 위의 작업을 동일하게 test데이터에도 진행
for i, file in enumerate(testFileList):
    # label, nm, tr, _ = file.split('_')
    label, _ = file.split('_')
    code, ext = _.split('.')

    new_img = load_img(path+'test\\'+file)
    arr_img = img_to_array(new_img)
    img = arr_img.reshape((1,)+arr_img.shape)
    if i == 0 :  # test 데이터
        container = img
        labels = word[label]
    else:
        container = np.vstack([container, img])
        labels = np.vstack([labels, word[label]])

xTest = container
yTest = np_utils.to_categorical(labels,5)   

In [60]:
print(container.shape, labels.shape)
print(xTrain.shape, yTrain.shape)
print(xTest.shape, yTest.shape)

(79, 288, 864, 3) (79, 1)
(79, 288, 864, 3) (79, 5)
(79, 288, 864, 3) (79, 5)


In [61]:
xTrain = xTrain.astype('float32')/255
xTest = xTest.astype('float32')/255

In [62]:
from keras.layers import *
from sklearn.metrics import accuracy_score

In [63]:
# 모델 구성
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(288,864,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [64]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(xTrain, yTrain, batch_size=10, epochs=200)

Epoch 1/200
8/8 [==============================] - 129s 16s/step - loss: 43.0097 - accuracy: 0.3038
Epoch 2/200
8/8 [==============================] - 120s 15s/step - loss: 1.5294 - accuracy: 0.3038
Epoch 3/200
8/8 [==============================] - 82s 10s/step - loss: 1.5066 - accuracy: 0.1899
Epoch 4/200
8/8 [==============================] - 69s 9s/step - loss: 1.4740 - accuracy: 0.2785
Epoch 5/200
8/8 [==============================] - 80s 10s/step - loss: 1.5002 - accuracy: 0.2532
Epoch 6/200
8/8 [==============================] - 83s 10s/step - loss: 1.4874 - accuracy: 0.2785
Epoch 7/200
8/8 [==============================] - 76s 10s/step - loss: 1.7092 - accuracy: 0.2658
Epoch 8/200
8/8 [==============================] - 79s 10s/step - loss: 1.4343 - accuracy: 0.2532
Epoch 9/200
8/8 [==============================] - 84s 10s/step - loss: 1.4225 - accuracy: 0.2785
Epoch 10/200
8/8 [==============================] - 82s 10s/step - loss: 1.4800 - accuracy: 0.1772
Epoch 11/200
8/8 

In [ ]:
result = model.predict_classes(xTest)

answer = []
for i in yTest:
    answer.append(np.argmax(i))

yTest_label = []
for y in yTest:
    yTest_label.append(np.argmax(y))

accuracy_score(yTest_label, result)